## Evitando sobreajuste por meio de regularização

In [1]:
# Python ≥3.5 é necessário
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 é necessário
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 é necessário
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

import numpy as np
import os

np.random.seed(42)

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

## Regularização pela técnica Dropout

In [3]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 11s 6ms/step - loss: 0.6106 - accuracy: 0.7985 - val_loss: 0.3831 - val_accuracy: 0.8706
Epoch 2/2
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4516 - accuracy: 0.8413 - val_loss: 0.3712 - val_accuracy: 0.8644


## Técnica Alpha Dropout

Variação da técnica dropout, que preserva o valor da média e desvio padrão dos valores originais de entrada, para fins de normalização.

In [4]:
tf.random.set_seed(42)
np.random.seed(42)

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 20
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
1719/1719 [==============================] - 11s 6ms/step - loss: 0.6659 - accuracy: 0.7589 - val_loss: 0.5884 - val_accuracy: 0.8472
Epoch 2/20
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5566 - accuracy: 0.7973 - val_loss: 0.5617 - val_accuracy: 0.8418
Epoch 3/20
1719/1719 [==============================] - 10s 6ms/step - loss: 0.5266 - accuracy: 0.8049 - val_loss: 0.4981 - val_accuracy: 0.8588
Epoch 4/20
1719/1719 [==============================] - 11s 7ms/step - loss: 0.5107 - accuracy: 0.8107 - val_loss: 0.4627 - val_accuracy: 0.8668
Epoch 5/20
1719/1719 [==============================] - 11s 6ms/step - loss: 0.4936 - accuracy: 0.8159 - val_loss: 0.4498 - val_accuracy: 0.8590
Epoch 6/20
1719/1719 [==============================] - 14s 8ms/step - loss: 0.4812 - accuracy: 0.8235 - val_loss: 0.4520 - val_accuracy: 0.8588
Epoch 7/20
1719/1719 [==============================] - 14s 8ms/step - loss: 0.4699 - accuracy: 0.8247 - val_loss: 0.4857 - val_acc